In [1]:
import sys
sys.path.insert(0, '..')

from formasaurus import features, evaluation
from formasaurus.storage import Storage, FORM_TYPES, FORM_TYPES_INV, load_html

### Available training data

In [2]:
storage = Storage("../formasaurus/data")
storage.check()
storage.print_type_counts()

Checking: |##########| 343/343 100% [elapsed: 00:01 left: 00:00, 176.81 iters/sec] OK
Annotated HTML forms:

177   search                    (s)
139   login                     (l)
111   other                     (o)
89    registration              (r)
50    password/login recovery   (p)
38    join mailing list         (m)
36    contact                   (c)

Total form count: 640


## Load training / evaluation data

We must be careful when splitting the dataset into training and evaluation parts: forms from the same domain should be in the same "bin". There could be several pages from the same domain, and these pages may have duplicate or similar forms (e.g. a search form on each page). If we put one such form in training dataset and another in evaluation dataset then the metrics will be too optimistic, and they can make us to choose wrong features/models. For example, train_test_split from scikit-learn shouldn't be used here.

As an approximation, data is sorted by a domain below (it is done in storage.get_Xy function), and then it is split into 2 parts. It means that no more than 1 domain is prone to overfitting (the one we're splitting at).

In [3]:
TRAIN_SIZE = 500

X, y = storage.get_Xy(verbose=True, leave=True)
X_train, X_test, y_train, y_test = X[:TRAIN_SIZE], X[TRAIN_SIZE:], y[:TRAIN_SIZE], y[TRAIN_SIZE:]

print("Train size: %d, test size: %d" % (len(y_train), len(y_test)))

Loading: |##########| 343/343 100% [elapsed: 00:01 left: 00:00, 178.82 iters/sec]
Train size: 500, test size: 140


## Ideas for useful features

### Search forms

* a single query field
* a field named "q" or "s"
* "search" in URL
* "search" in submit button text (submit value)
* "search" in form css class or id
* no password field
* method == GET?

### Login forms

* username or email and password
* 2 passwords - likely not a login form
* "login" or "sign in" (or variations) in URL
* "login" or "sign in" (or variations) in form css class or id
* "login" or "sign in" in submit button text
* "Remember me" checkbox (or any single checkbox)
* no select elements
* no textarea elements
* openid?
* method == POST

### Registration forms

* 2 passwords 
* "register" / "sign up" in URL, form css class / id or submit button text
* "agree" checkbox
* email
* username
* method == POST

### Contact forms

* feedback in URL/class
* textarea
* "Send" button
* email
* method == POST

### Password reset

* a single email or username field
* "password" in URL/css class/ submit button text

### Join Mailing List

* a single email field
* subscribe/join/newsletter words
* a short form

The main problem with "join mailing list" forms is to distinguish them from search forms.

## How to handle them

Instead of hardcoding the features above many of them are generalized. For exmaple, instead of writing "search in URL" we extract all 5-character substrings from the URL and use "`urlsubstring<N>` in URL" as features. There are some disadvantages in this approach, but it provides a good starting point.

The feature extractors are stored in formtype.features module.

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

In [6]:
%%time
reload(features)
from formasaurus.model import _create_feature_union
# ======= define the model ========

# features should be the same as in formtype.model
FEATURES = [
    (
        "bias",
        features.Bias(),
        DictVectorizer(),
    ),
    (
        "form elements",
        features.FormElements(),
        DictVectorizer()
    ),
    (
        "<input type=submit value=...>",
        features.SubmitText(),
        CountVectorizer(ngram_range=(1,2), min_df=1, binary=True)
    ),
    # (
    #     "<form> TEXT </form>",
    #     features.FormText(),
    #     TfidfVectorizer(ngram_range=(1,2), min_df=5, stop_words='english', binary=True)
    # ),

    (
        "<a> TEXT </a>",
        features.FormLinksText(),
        TfidfVectorizer(ngram_range=(1,2), min_df=4, stop_words={'and', 'or', 'of'}, binary=True)
    ),
    (
        "<label> TEXT </label>",
        features.FormLabelText(),
        TfidfVectorizer(ngram_range=(1,2), min_df=3, stop_words="english", binary=True)
    ),

    (
        "<form action=...>",
        features.FormUrl(),
        TfidfVectorizer(ngram_range=(5,6), min_df=4, analyzer="char_wb", binary=True)
    ),
    (
        "<form class=... id=...>",
        features.FormCss(),
        TfidfVectorizer(ngram_range=(4,5), min_df=3, analyzer="char_wb", binary=True)
    ),
    (
        "<input class=... id=...>",
        features.FormInputCss(),
        TfidfVectorizer(ngram_range=(4,5), min_df=5, analyzer="char_wb", binary=True)
    ),
    (
        "<input name=...>",
        features.FormInputNames(),
        TfidfVectorizer(ngram_range=(5,6), min_df=3, analyzer="char_wb", binary=True)
    ),
    (
        "<input title=...>",
        features.FormInputTitle(),
        TfidfVectorizer(ngram_range=(5,6), min_df=3, analyzer="char_wb", binary=True)
    ),
]



# clf = SGDClassifier(
#     penalty='elasticnet', 
#     loss='log', 
#     alpha=0.0002,
#     fit_intercept=False, 
#     shuffle=True, 
#     random_state=0,
#     n_iter=50,
# )
clf = LogisticRegression(penalty='l2', C=5, fit_intercept=False, random_state=0)
# clf = LinearSVC(C=0.5, random_state=0, fit_intercept=False)

model = make_pipeline(
    _create_feature_union(FEATURES), 
    clf,
)
    
evaluation.print_metrics(model, X, y, X_train, X_test, y_train, y_test, ipython=True)


Classification report (500 training examples, 140 testing examples):

                         precision    recall  f1-score   support

                contact       1.00      0.88      0.93         8
                  login       0.96      0.96      0.96        28
      join mailing list       0.86      0.50      0.63        12
                  other       0.68      1.00      0.81        19
password/login recovery       1.00      0.92      0.96        13
           registration       1.00      0.78      0.88        18
                 search       0.95      1.00      0.98        42

            avg / total       0.92      0.91      0.90       140

Active features: 31388 out of possible 31388

Confusion matrix (rows=>true values, columns=>predicted values):


,contact,login,join mailing list,other,password/login recovery,registration,search
contact,7,0,0,1,0,0,0
login,0,27,0,1,0,0,0
join mailing list,0,0,6,4,0,0,2
other,0,0,0,19,0,0,0
password/login recovery,0,0,0,1,12,0,0
registration,0,1,1,2,0,14,0
search,0,0,0,0,0,0,42



Running cross validation...
10-fold cross-validation F1: 0.900 (±0.087)  min=0.828  max=0.953
CPU times: user 8.43 s, sys: 118 ms, total: 8.55 s
Wall time: 8.71 s


## Check what classifier learned

For linear classifiers like Logistic Regression or an SVM without a kernel we can check coefficient values to understand better how the decision is made. 

For correlated features weight will be spread across all correlated features, so just checking coefficients is not enough, but looking at them is useful anyways.

In [7]:
evaluation.print_informative_features(FEATURES, clf, 25)

contact
+3.3853:                  form elements  has <textarea>
+1.9748:                  <a> TEXT </a>  contact
+1.9487:                  <a> TEXT </a>  contact us
+1.8169:                  <a> TEXT </a>  us
+1.5304:                  <a> TEXT </a>  privacy
+1.1378:        <form class=... id=...>  back
+1.0006:  <input type=submit value=...>  отправить
+0.6777:  <input type=submit value=...>  send
+0.6696:  <input type=submit value=...>  submit
+0.5955:  <input type=submit value=...>  értesítőt küldök
+0.5955:  <input type=submit value=...>  neki
+0.5955:  <input type=submit value=...>  küldök neki
+0.5955:  <input type=submit value=...>  küldök
+0.5955:  <input type=submit value=...>  értesítőt
+0.5744:          <label> TEXT </label>  message
+0.5703:          <label> TEXT </label>  address
+0.5452:          <label> TEXT </label>  имя
+0.5386:          <label> TEXT </label>  phone
+0.5292:          <label> TEXT </label>  city
+0.5117:        <form class=... id=...>  _form
+0.5117:    

## Compare results with "loginform" library

It is not possible to compare the results with "loginform" library directly because loginform

* always tries to return a login form even if the score is low;
* only detects login forms;
* in case of several forms returns a single form with the best score instead of deciding for each form whether to return it or not.

So we used two approaches:

1. Use `loginform._form_score` with different thresholds; assume that if score is greater than or equal to a threshold `loginform` detected a login form.
2. Train the same model, but using features from loginform library (weights will be learned instead of being hardcoded as 'score' increments/decrements).



### 1. loginform scores + thresholds

* **score >= -100** means "simply treat all forms as login forms".

* **score >= 0** all (or most) login forms are captured, but there are many false positives. 
  It is only slightly better than treating all forms as login forms.

* **score >= 10** F1 score is the best among all thresholds, 
  but the quality is significantly worse than F1 of ML-based models.
  
* **score >= 20** ~90% of detected login forms are correct, but most 
  login forms are not detected. Also, ~90% number is still lower than what ML-based models give us.


In [8]:
%%time 
import loginform


def labels_to_binary(y):
    """ Convert labels to 2-classes: login forms and non-login forms """
    return [tp == 'l' for tp in y]

    
def predict_loginform(X, threshold):
    """
    Return if forms are login or not using loginform
    library scores and a threshold.
    """
    return [
        (loginform._form_score(form) >= threshold)
        for form in X
    ]


def print_threshold_metrics(X_test, y_test, threshold):
    y_test = labels_to_binary(y_test)
    y_pred = predict_loginform(X_test, threshold)

    precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, pos_label=True)
    print(
        "score >= %4d:    precision = %0.3f    recall = %0.3f    F1 = %0.3f" % (
        threshold, precision[1], recall[1], f1[1]
    ))


for threshold in [-100, -10, 0, 10, 20, 30]:
    print_threshold_metrics(X_test, y_test, threshold)

score >= -100:    precision = 0.200    recall = 1.000    F1 = 0.333
score >=  -10:    precision = 0.233    recall = 0.964    F1 = 0.375
score >=    0:    precision = 0.409    recall = 0.964    F1 = 0.574
score >=   10:    precision = 0.710    recall = 0.786    F1 = 0.746
score >=   20:    precision = 0.875    recall = 0.250    F1 = 0.389
score >=   30:    precision = 0.000    recall = 0.000    F1 = 0.000
CPU times: user 256 ms, sys: 2.65 ms, total: 259 ms
Wall time: 260 ms


/Users/kmike/envs/fortia/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 2. Use loginform features, but autodetect scores

The following ML-based model is trained using original loginform features (conditions used to increase or decrease the score). Roughly speaking, it uses the same information as loginform library, but instead of hardcoding `score += 10` and `score -= 10` the numbers are adjusted based on training data.

Note that the login form detection quality is significantly better than the quality of threshold-based model; it is only slightly worse than the quality of a "full" forms detection model. This means original loginform features are quite good at detecting login forms. But for other form types these features are not enough: other scores are bad.

In [9]:
%%time

LOGINFORM_FEATURES = [
    ('bias', features.Bias(), DictVectorizer()),
    ('loginform', features.OldLoginformFeatures(), DictVectorizer())
]
# loginform_clf = LinearSVC(C=0.5, fit_intercept=False)
loginform_clf = LogisticRegression(penalty='l2', C=5, fit_intercept=False, random_state=0)

model = make_pipeline(
    _create_feature_union(LOGINFORM_FEATURES), 
    loginform_clf,
)

evaluation.print_metrics(model, X, y, X_train, X_test, y_train, y_test, ipython=True)
evaluation.print_informative_features(LOGINFORM_FEATURES, loginform_clf, 25)


Classification report (500 training examples, 140 testing examples):

                         precision    recall  f1-score   support

                contact       0.88      0.88      0.88         8
                  login       0.93      0.96      0.95        28
      join mailing list       0.00      0.00      0.00        12
                  other       0.61      1.00      0.76        19
password/login recovery       0.00      0.00      0.00        13
           registration       0.87      0.72      0.79        18
                 search       0.65      0.88      0.75        42

            avg / total       0.63      0.74      0.67       140

Active features: 56 out of possible 56

Confusion matrix (rows=>true values, columns=>predicted values):


,contact,login,join mailing list,other,password/login recovery,registration,search
contact,7,0,0,1,0,0,0
login,0,27,0,1,0,0,0
join mailing list,0,0,0,0,0,0,12
other,0,0,0,19,0,0,0
password/login recovery,0,0,0,6,0,0,7
registration,1,2,0,1,0,13,1
search,0,0,0,3,0,2,37



Running cross validation...
10-fold cross-validation F1: 0.647 (±0.117)  min=0.553  max=0.728
contact
+3.8461:                      loginform  typecount_text_gt1
+2.5897:                      loginform  typecount_password_0
+0.9392:                      loginform  typecount_text_0
-0.5198:                      loginform  typecount_radio_gt0
-1.2640:                      loginform  typecount_checkbox_gt1
-1.4278:                      loginform  typecount_password_eq1
-2.9678:                      loginform  2_or_3_inputs
-6.5143:                           bias  bias
--------------------------------------------------------------------------------
login
+4.6784:                      loginform  typecount_password_eq1
+0.6345:                      loginform  typecount_text_0
+0.0009:                      loginform  2_or_3_inputs
-1.6849:                      loginform  typecount_password_0
-1.7660:                      loginform  typecount_text_gt1
-1.9949:                           bias  

/Users/kmike/envs/fortia/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
